# METOD algorithm - Minimum of several quadratic forms

# 1. Import libraries

The following libraries are required to run the METOD Algorithm.

In [ ]:
import numpy as np 
from numpy import linalg as LA
import pandas as pd

import metod as mt
from metod import objective_functions as mt_obj

# 2. Define function and gradient

Minimum of several quadratic forms:
 \begin{equation}
 \label{eq:model}
f(x_n^{(k)})=\min_{1\le p \le P} \frac{1}{2} (x_n^{(k)}-x_{0p})^T A_p^T \Sigma_p A_p (x_n^{(k)}-x_{0p}),
\end{equation}
where $x_n^{(k)}$ is the $n$-th point after $k$ iterations of anti-gradient descent, $P$ is the number of minima; $A_p$ is a random rotation
matrix of size $d\times d$; $\Sigma_p$ is a diagonal positive
definite matrix of size $d\times d$ with smallest and largest
eigenvalues $\lambda_{min}$ and $\lambda_{max}$ respectively;
$x_{0p} \in \mathfrak{X}$ and $p=1,...,P$.

Note : Anti-gradient descent iterations are terminated at the smallest $k=K_n$ such that $\nabla f(x_n^{(k)}) < \delta$, where $\delta$ is some small positive constant. 


In [ ]:
f = mt_obj.several_quad_function
g = mt_obj.several_quad_gradient

# 3. Defining parameters

The following parameters are required to derive $A_p$ ($p=1,...,P$) ; $\Sigma_p$ ($p=1,...,P$) and $x_{0p}$ ($p=1,...,P$).


•d: dimension

•P: number of minima

•lambda_1: smallest eigenvalue of $\Sigma_p$ ($p=1,...,P$)

•lambda_2: largest eigenvalue of $\Sigma_p$ ($p=1,...,P$)

In order to replicate results, we will control the pseudo-random number generator seed, so that the same random objective function and random starting points $x_n^{(0)}$ $(n=1,...,1000)$ will be generated each time the code is run. The random seed number will be set to 90.

In [ ]:
d = 50
P = 5
lambda_1 = 1
lambda_2 = 10
seed = 90

In [ ]:
np.random.seed(seed)
store_x0, matrix_combined = mt_obj.function_parameters_several_quad(P, d, lambda_1, lambda_2)

Where,

•store_x0: $x_{0p}$ ($p=1,...,P$)

•matrix_combined: $A_p^T \Sigma_p A_p$ ($p=1,...,P$)


In [ ]:
args = P, store_x0, matrix_combined

# 4. Run METOD Algorithm

In [ ]:
(discovered_minimizers,
 number_minimizers,
 func_vals_of_minimizers,
 excessive_no_descents,
 starting_points) = mt.metod(f, g, args, d)

# 5. Results of the METOD Algorithm

Total number of minimizers found:

In [ ]:
number_minimizers

Positions of minimizers:

In [ ]:
discovered_minimizers

Function values of minimizers:

In [ ]:
func_vals_of_minimizers

Total number of excessive descents:

In [ ]:
excessive_no_descents

# 6. Save results to csv file (optional)

The below csv files will be saved to the same folder which contains the METOD Algorithm - Minimum of several quadratic forms notebook.

Rows in discovered_minimizers_d_%s_p_%s_quad.csv represent discovered minimizers. The total number of rows will be the same as the value for number_minimizers.

In [ ]:
np.savetxt('discovered_minimizers_d_%s_p_%s_quad.csv' % (d, P), discovered_minimizers, delimiter=",")

Each row in func_vals_discovered_minimizers_d_%s_p_%s_quad.csv represents the function value of each discovered minimizer. The total number of rows will be the same as the value for number_minimizers.

In [ ]:
np.savetxt('quad_func_vals_discovered_minimizers_d_%s_p_%s_quad.csv' % (d, P), func_vals_of_minimizers, delimiter=",")

summary_table_d_%s_p_%s_quad.csv will contain the total number of minimizers discovered and the total number of excessive descents.

In [ ]:
summary_table = pd.DataFrame({
"Total number of unique minimizers": [number_minimizers],
"Extra descents": [excessive_no_descents]})
summary_table.to_csv('summary_table_d_%s_p_%s_quad.csv' % (d, P))

# 7. Test results (optional)

This test can only be used for the minimum of several quadratic forms function.

To check each discovered minimizer is unique, we do the following:

For each minimizer $x_l^{(K_l)}$ ($l=1,...,L$) 

\begin{equation}
p_l = {\rm argmin}_{1\le p \le P} \frac{1}{2} (x_l^{(K_l)}-x_{0p})^T A_p^T \Sigma_p A_p (x_l^{(K_l)}-x_{0p})
\end{equation}

We must have $p_l$ is different for all $x_l^{(K_l)}$ ($l=1,...,L$), in order for minimizers to be unique.

We also test that $\|x_l^{(K_l)} - x_{0p_l}\| \text{  is small}$.


In [ ]:
def calc_pos(point, d,  p, store_x0, matrix_combined):
    """Position (p_l) of the minimum of the function""" 
    function_values = np.zeros((p))
    for i in range(p):
        function_values[i] = 0.5 * np.transpose(point - store_x0[i]) @ matrix_combined[i] @ (point - store_x0[i])
    position_minimum = np.argmin(function_values)
    return position_minimum 

In [ ]:
"""Store values from calc_pos function""" 
norms_with_minimizer = np.zeros((number_minimizers))
pos_list = np.zeros((number_minimizers))
for j in range(number_minimizers):
    pos = calc_pos(discovered_minimizers[j], d, *args)
    pos_list[j] = pos
    norms_with_minimizer[j] = LA.norm(discovered_minimizers[j]- store_x0[pos])

${\max}_{1\le l \le L}  \|x_l^{(K_l)}-x_{0p_l}\|$ should be small

In [ ]:
np.max(norms_with_minimizer)

Ensure that the number of unique minimizers is $L$

In [ ]:
np.unique(pos_list).shape[0] == number_minimizers